In [3]:
import pandas as pd
import json
from pathlib import Path

def make_trip_shape_map(trip_id,
                        trips_file='trips.txt',
                        shapes_file='shapes.txt',
                        output_html='trip_shape_map.html'):
    """
    Given a GTFS trips.txt and shapes.txt, generate an HTML map showing the shape for a trip_id.
    """

    # --- Load trips.txt ---
    trips = pd.read_csv(trips_file, dtype=str)
    if 'trip_id' not in trips.columns or 'shape_id' not in trips.columns:
        raise ValueError("trips.txt must have 'trip_id' and 'shape_id' columns")

    # Find shape_id for this trip
    row = trips.loc[trips['trip_id'] == str(trip_id)]
    if row.empty:
        raise ValueError(f"Trip ID {trip_id} not found in {trips_file}")
    shape_id = row.iloc[0]['shape_id']
    print(f"Trip {trip_id} uses shape_id {shape_id}")

    # --- Load shapes.txt ---
    shapes = pd.read_csv(shapes_file, dtype=str)
    required_cols = {'shape_id', 'shape_pt_lat', 'shape_pt_lon', 'shape_pt_sequence'}
    if not required_cols.issubset(shapes.columns):
        raise ValueError(f"shapes.txt must have columns: {required_cols}")

    # Filter for this shape_id and sort by sequence
    shape = shapes[shapes['shape_id'] == shape_id].copy()
    shape['shape_pt_sequence'] = shape['shape_pt_sequence'].astype(int)
    shape.sort_values('shape_pt_sequence', inplace=True)

    # Extract coordinates
    coords = shape[['shape_pt_lat', 'shape_pt_lon']].astype(float).values.tolist()
    coords_js = json.dumps(coords)

    # Map center
    center_lat = shape['shape_pt_lat'].astype(float).mean()
    center_lon = shape['shape_pt_lon'].astype(float).mean()

    # --- HTML template ---
    html = f"""<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8" />
  <title>Trip {trip_id} Shape Map</title>
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <link rel="stylesheet" href="https://unpkg.com/leaflet@1.9.4/dist/leaflet.css" />
  <style>
    html, body, #map {{ height: 100%; margin: 0; padding: 0; }}
  </style>
</head>
<body>
  <div id="map"></div>
  <script src="https://unpkg.com/leaflet@1.9.4/dist/leaflet.js"></script>
  <script>
    const coords = {coords_js};
    const map = L.map('map').setView([{center_lat}, {center_lon}], 13);
    L.tileLayer('https://{{s}}.tile.openstreetmap.org/{{z}}/{{x}}/{{y}}.png', {{
      maxZoom: 19,
      attribution: '© OpenStreetMap contributors'
    }}).addTo(map);

    const poly = L.polyline(coords, {{color: 'blue', weight: 5}}).addTo(map);
    map.fitBounds(poly.getBounds());

    // Mark start and end
    if (coords.length > 0) {{
      L.marker(coords[0]).addTo(map).bindPopup('Start');
      L.marker(coords[coords.length - 1]).addTo(map).bindPopup('End');
    }}
  </script>
</body>
</html>
"""
    Path(output_html).write_text(html, encoding='utf-8')
    print(f"✅ Map saved as: {output_html}")
    return output_html


# --- Example usage ---
# Change this to your actual trip ID
trip_id = "-Q2gVX9GgVSEtVr-yv6Nf-07:00:00"
make_trip_shape_map(trip_id, 'trips.txt', 'shapes.txt', 'trip_shape_map.html')


Trip -Q2gVX9GgVSEtVr-yv6Nf-07:00:00 uses shape_id -Q2gVX9GgVSEtVr-yv6Nf_Shape
✅ Map saved as: trip_shape_map.html


'trip_shape_map.html'